# Background: Classical TEP Model
- Implementation of the classical TEP model on the Garver 6-bus system.

## Load Packages

In [1]:
using JuMP, HiGHS
using CSV, DataFrames

## Load Data

In [2]:
buses = CSV.read(joinpath("..","data","Garver","buses.csv"),DataFrame)
generators = CSV.read(joinpath("..","data","Garver","generators.csv"),DataFrame)
lines = CSV.read(joinpath("..","data","Garver","lines.csv"),DataFrame);

## Classical TEP Formulation

$\begin{align}
    &\min \quad Y \times 8760 \times \left[\sum_{g\in G} Cost^{en}_g p_g + \sum_{i\in N} Cost^{voll} ens_i\right] + \sum_{ij\in C^{ac}} Cost^{ac}_{ij}(-\mathrm{A}_{ij} + \sum_{k\in K} \alpha_{ij,k}) \notag\\
    &\text{subject to} \notag\\
    &\sum_{g\in G_i} p_g + \sum_{ij\in C^{ac}} \sum_{k\in K}(f^{ac}_{ji,k} - f^{ac}_{ij,k}) = Load_i \quad \forall i\in N \notag\\
    &-M(1-\alpha_{ij,k}) \leq f^{ac}_{ij,k} - S_{base}\frac{\theta_i - \theta_j}{X_{ij}} \leq M(1-\alpha_{ij,k}) \quad \forall k\in K, ij\in C^{ac}\\
    &-\alpha_{k,ij}Cap_{ij} \leq f_{ij,k}^{ac} \leq \alpha_{k,ij}Cap_{ij} \quad \forall k\in K, ij\in C^{ac} \notag\\
    &P_g^{min} \leq p_g \leq P_g^{max} \quad \forall g\in G \notag\\
    &\sum_{k\in K} \alpha_{ij,k} \geq \mathrm{A}_{ij} \quad \forall ij\in C^{ac} \notag\\
    &\alpha_{ij,k} \in \{0,1\} \quad \forall k\in K, ij\in C^{ac} \notag\\
    &k\in K, K = \{1,2,3,4\} \notag
\end{align}$

## Model Classical TEP using JuMP

In [3]:
function classicaltep(buses, generators, lines, Y, M, max_k, VOLL, epsilon, S_base, silent_mode=false)
    # SETS
    N = buses.id
    G = generators.id
    C_ac = lines.id
    K = 1:max_k

    # INIT MODEL
    model = Model()
    set_optimizer(model, HiGHS.Optimizer)
    set_optimizer_attribute(model, "mip_rel_gap", epsilon)
    if silent_mode
        set_silent(model)
    end

    # VARIABLES
    @variables(model, begin
        p[G] >= 0
        ens[N] >= 0
        f_ac[C_ac,K]
        theta[N]
        alpha[C_ac,K], Bin
    end)
    fix(theta[1], 0, force=true)

    # OBJECTIVE FUNCTION
    @expression(model, Cost_op, Y * 8760 * sum(generators[g, :cost] * p[g] for g in G))
    @expression(model, Cost_ens, Y * 8760 * sum(VOLL * ens[i] for i in N))
    @expression(model, Cost_ac, 
        sum(lines[l,:cost] * 1e6 * (sum(alpha[l,k] for k in K) - lines[l,:a]) for l in C_ac))
    @objective(model, Min, Cost_op + Cost_ens + Cost_ac)

    # CONSTRAINTS
    @constraint(model, cPowerBalance[i in N], 
        sum(p[g] for g in generators[generators.node.==i,:id]) - 
        sum(f_ac[l,k] for l in lines[lines.from.==i,:id] for k in K) + 
        sum(f_ac[l,k] for l in lines[lines.to.==i,:id] for k in K) - ens[i] == buses[i,:load])
    @constraint(model, cPowerFlowPositiveBigM[l in C_ac, k in K],
        f_ac[l,k] - S_base*(1/lines[l,:x])*(theta[lines[l,:from]]-theta[lines[l,:to]]) <= 
        M*(1-alpha[l,k]))
    @constraint(model, cPowerFlowNegativeBigM[l in C_ac, k in K],
        f_ac[l,k] - S_base*(1/lines[l,:x])*(theta[lines[l,:from]]-theta[lines[l,:to]]) >= 
        -M*(1-alpha[l,k]))
    @constraint(model, cLineThermalLimitPositive[l in C_ac, k in K], 
        f_ac[l,k] <= alpha[l,k] * lines[l,:cap])
    @constraint(model, cLineThermalLimitNegative[l in C_ac, k in K], 
        f_ac[l,k] >= -alpha[l,k] * lines[l,:cap])
    @constraint(model, cUnitThermalLimitMax[g in G],
        p[g] <= generators[g,:pmax])
    @constraint(model, cUnitThermalLimitMin[g in G],
        p[g] >= generators[g,:pmin])
    @constraint(model, cExistingLines[l in C_ac],
        sum(alpha[l,k] for k in K) >= lines[l,:a])
    
    # SOLVE MODEL
    optimize!(model)

    # RETURN SOLUTIONS
    return(
        Cost_op = value(Cost_op),
        Cost_ens = value(Cost_ens),
        Cost_ac = value(Cost_ac),
        p = value.(p).data,
        f_ac = value.(f_ac).data,
        theta = value.(theta).data,
        alpha = value.(alpha).data,
        ens = value.(ens).data,
        Obj = objective_value(model)
    )
end

classicaltep (generic function with 2 methods)

## Parameters Setting

In [4]:
M = 1000
max_k = 4
VOLL = 1000
epsilon = 0.01
S_base = 100
silent_mode=true;

## Solve TEP for Y=2

In [5]:
sol_y2 = classicaltep(buses, generators, lines, 2, M, max_k, VOLL, epsilon, S_base, silent_mode);

## Investment Decisions for Y=2

In [6]:
DataFrame(
    [lines.id lines.from lines.to round.(sum(sol_y2.alpha, dims=2) - lines[:,:a])], [:line_id, :from, :to, :new_lines]
    )

Row,line_id,from,to,new_lines
,Float64,Float64,Float64,Float64
1,1.0,1.0,2.0,0.0
2,2.0,1.0,4.0,-0.0
3,3.0,1.0,5.0,1.0
4,4.0,2.0,3.0,0.0
5,5.0,2.0,4.0,0.0
6,6.0,2.0,6.0,3.0
7,7.0,3.0,5.0,0.0
8,8.0,4.0,6.0,0.0


## Operation, Investment, ENS, and Total Cost for Y=2 in $M

In [7]:
(sol_y2.Cost_op, sol_y2.Cost_ac, sol_y2.Cost_ens, sol_y2.Obj)./1e6

(227.14164705882348, 109.99999999998678, 0.0, 337.14164705881024)

## Solve TEP for Y=10

In [8]:
sol_y10 = classicaltep(buses, generators, lines, 10, M, max_k, VOLL, epsilon, S_base, silent_mode);

## Investment Decisions for Y=10

In [9]:
DataFrame(
    [lines.id lines.from lines.to round.(sum(sol_y10.alpha, dims=2) - lines[:,:a])], [:line_id, :from, :to, :new_lines]
    )

Row,line_id,from,to,new_lines
,Float64,Float64,Float64,Float64
1,1.0,1.0,2.0,0.0
2,2.0,1.0,4.0,0.0
3,3.0,1.0,5.0,1.0
4,4.0,2.0,3.0,1.0
5,5.0,2.0,4.0,0.0
6,6.0,2.0,6.0,4.0
7,7.0,3.0,5.0,0.0
8,8.0,4.0,6.0,2.0


## Operation, Investment, ENS, and Total Cost for Y=10 in $M

In [10]:
(sol_y10.Cost_op, sol_y10.Cost_ac, sol_y10.Cost_ens, sol_y10.Obj)./1e6

(814.2628571428511, 220.00000000000023, 0.0, 1034.2628571428515)

## Comparison between Y=2 and Y=10

In [11]:
summary = DataFrame(
    [collect((sol_y2.Cost_op, sol_y2.Cost_ac, sol_y2.Cost_ens, sol_y2.Obj)./1e6)';
    collect((sol_y10.Cost_op, sol_y10.Cost_ac, sol_y10.Cost_ens, sol_y10.Obj)./1e6)'],
    [:Cost_op, :Cost_ac, :Cost_ens, :Total])
insertcols!(summary,1,:Y=>[2,10])
summary

Row,Y,Cost_op,Cost_ac,Cost_ens,Total
,Int64,Float64,Float64,Float64,Float64
1,2,227.142,110.0,0.0,337.142
2,10,814.263,220.0,0.0,1034.26
